<p style="background-color:#90EE90;color:black;font-size:45px;text-align:center;border-radius:9px 9px;font-weight:bold;border:2px black;">Titanic - SVM and RF Parameters and Deep Learning</p>

In [ ]:
# Project Planning --
# 1. The first thing we want to do is an inspection of the data
# 2. Cleaning the data (NaNs, incorrect values, etc)
# 3. Transform the data and create dummy variables
# 4. The plan is to run both a RandomForest and SVM model
# 5. We want the models to be tuned to their best parameters so we will attempt to write a script to do that
# 6. We will then see the best feature parameters of both the models to use for deep learning
# 7. The deep model will be handled with Keras. We want to first explore the different optimizers (We will run a model using each one and see the different accuracies)
# 8. We will then use the best optimizer to create the final fit using K-fold cross-validation over 10 cycles.

## <p style="background-color:#90EE90;color:black;font-size:30px;text-align:center;border-radius:12px 10px;border:2px;">   Here is the whole dataset </p>


<font size="4">The dataset contains ten categories/variables with 891 passenger details. Survived is the response variable for the study. We are also given a seperate "test" set of data that does not give a survival result </font>
<p></p>


<table style="border-collapse: collapse;font-size: 15px; width:800px;">
  <tr>
    <th style="background-color:#D3DBDD;">Variable Name </th>
    <th style="width:500px; background-color:#D3DBDD;">Description</th>
    <th style="background-color:#D3DBDD;">Type</th>
  </tr>
  <tr>
    <td>survival</td>
    <td>Did Survive the incident?</td>
    <td>Categoricol</td>
  </tr>
  <tr>
    <td>pclass </td>
    <td>Class of the ticket</td>
    <td>Categoricol</td>
  </tr>
  <tr>
    <td>sex </td>
    <td>Gender </td>
    <td>Categoricol</td>
  </tr>
  <tr>
    <td>Age </td>
    <td>Age of the passenger</td>
    <td>Numeric</td>
  </tr>
  <tr>
    <td>sibsp </td>
    <td>no of siblings / spouses aboard the Titanic </td>
    <td>Numeric</td>
  </tr>
  <tr>
    <td>parch</td>
    <td>no of parents / children aboard the Titanic</td>
    <td>Numeric</td>
  </tr>
  <tr>
    <td>ticket</td>
    <td>Unique ticket number</td>
    <td>Categoricol</td>
  </tr>
  <tr>
    <td>fare</td>
    <td>Passenger fare </td>
    <td>Numeric</td>
  </tr>
  <tr>
    <td>cabin</td>
    <td>cabin number </td>
    <td>Categoricol</td>
  </tr>
  <tr>
    <td>Embarked</td>
    <td>Port of Embarkation</td>
    <td>Categoricol</td>
  </tr>
</table>


## <p style="background-color:#90EE90;color:black;font-size:30px;text-align:center;border-radius:12px 10px;border:2px;"> 1. Data Inspection and Exploration </p>

<font size="4"> First we are going to do some simple data exploration, getting a feel for the different values used. </font>
<p></p>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

gender = pd.read_csv('../input/titanic/gender_submission.csv')

train_data = pd.read_csv('../input/titanic/train.csv')
test_data = pd.read_csv('../input/titanic/test.csv')

train_data['train'] = 1 # here we are defining a column so that we can concatenate the test/train data
test_data['train'] = 0
test_data['Survived'] = np.NaN
all_data = pd.concat([train_data,test_data])

%matplotlib inline
all_data.columns # check the columns

In [ ]:
# checking for dtype, column name, and seeing missing values
train_data.info()

In [ ]:
# getting the descriptive statistics
train_data.describe()

In [ ]:
#splitting data into numeric and categorical
dfnum = train_data[['Age','SibSp','Parch','Fare','Survived','Pclass']]
dfcat = train_data[['Sex','Ticket','Cabin','Embarked']]

In [ ]:
dfnum.describe()

In [ ]:
dfcat.describe()

In [ ]:
# here is a heatmap of the correlations between the 4 numeric variables
# as we can see there is a relationship between parch+sibsp (num of parents + num of siblings)
print(dfnum.corr())
sns.heatmap(dfnum.corr())

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'
dfnum.Age = dfnum.Age.fillna(dfnum.Age.mean())
dfnum.Fare = dfnum.Fare.fillna(dfnum.Fare.median())

In [ ]:
plt.figure(figsize=[16,12])
from scipy import stats


plt.subplot(231)
sns.histplot(all_data['Age'],kde=True)
plt.title('Age')
plt.xlabel(f'Shapiro test: {round(stats.shapiro(dfnum["Age"]).pvalue,6)}')

plt.subplot(232)
sns.histplot(all_data['SibSp'],kde=True)
plt.title('SibSp')
plt.xlabel(f'Shapiro test: {np.round(stats.shapiro(dfnum["SibSp"]).pvalue,6)}')

plt.subplot(233)
sns.histplot(all_data['Parch'],kde=True)
plt.title('Parch')
plt.xlabel(f'Shapiro test: {np.round(stats.shapiro(dfnum["Parch"]).pvalue,6)}')

plt.subplot(234)
sns.histplot(all_data['Survived'],kde=True)
plt.title('Survived')
plt.xlabel(f'Shapiro test: {np.round(stats.shapiro(dfnum["Survived"]).pvalue,6)}')

plt.subplot(235)
sns.histplot(all_data['Pclass'],kde=True)
plt.title('Pclass')
plt.xlabel(f'Shapiro test: {np.round(stats.shapiro(dfnum["Pclass"]).pvalue,6)}')



In [ ]:
# printing out some stacked histograms and some box plots for age and fare

plt.figure(figsize=[16,12])

# the fare data looks to be fairly skewed and we know from the shapiro test that it's not parametric
plt.subplot(231)
plt.boxplot(x=dfnum['Fare'], showmeans = True, meanline = True)
plt.title('Fare Boxplot')
plt.ylabel('Fare ($)')

# the same can be said for the age data - its looking likely that we will use either median or mode to fill in NA values
plt.subplot(232)
plt.boxplot(dfnum['Age'], showmeans = True, meanline = True)
plt.title('Age Boxplot')
plt.ylabel('Age (Years)')

# just a quick look to see if there is any obvious visual correlation between fare and survival
plt.subplot(233)
plt.hist(x = [dfnum[dfnum['Survived']==1]['Fare'], dfnum[dfnum['Survived']==0]['Fare']], 
         stacked=True, color = ['g','r'],label = ['Survived','Dead'])
plt.title('Fare Histogram by Survival')
plt.xlabel('Fare ($)')
plt.ylabel('# of Passengers')
plt.legend()

# and doing the same for age
plt.subplot(234)
plt.hist(x = [dfnum[dfnum['Survived']==1]['Age'], dfnum[dfnum['Survived']==0]['Age']], 
         stacked=True, color = ['g','r'],label = ['Survived','Dead'])
plt.title('Age Histogram by Survival')
plt.xlabel('Age (Years)')
plt.ylabel('# of Passengers')
plt.legend()



In [ ]:
dfcat.columns

In [ ]:
plt.figure(figsize=[16,12])
from scipy import stats


plt.subplot(231)
sns.barplot(x=dfcat['Sex'].value_counts().index,y=dfcat['Sex'].value_counts()).set(title='Sex')
plt.subplot(232)
sns.barplot(x=dfcat['Ticket'].value_counts().index,y=dfcat['Ticket'].value_counts()).set(title='Ticket')
plt.subplot(233)
sns.barplot(x=dfcat['Cabin'].value_counts().index,y=dfcat['Cabin'].value_counts()).set(title='Cabin')
plt.subplot(234)
sns.barplot(x=dfcat['Embarked'].value_counts().index,y=dfcat['Embarked'].value_counts()).set(title='Embarked')

# now looking at some of the categorical data descriptives


In [ ]:
# noticed that some passengers had multiple rooms, thought it was worth exploring separately
train_data['cabin_multiple'] = train_data.Cabin.apply(lambda x: 0 if pd.isna(x) else len(x.split(' ')))
train_data['cabin_multiple'].value_counts()

In [ ]:
plt.figure(figsize=[16,12])

plt.subplot(231)
plt.hist(x = [train_data[train_data['Survived']==1]['cabin_multiple'], train_data[train_data['Survived']==0]['cabin_multiple']], 
         stacked=True, color = ['g','r'],label = ['Survived','Dead'])
plt.title('cabin_multiple Histogram by Survival')
plt.xlabel('# of Cabins')
plt.ylabel('# of Passengers')
plt.legend()


pd.pivot_table(train_data, index = 'Survived', columns = 'cabin_multiple', values = 'Ticket', aggfunc = 'count')



## <p style="background-color:#90EE90;color:black;font-size:30px;text-align:center;border-radius:12px 10px;border:2px;"> 2. Data Cleaning and Preprocessing </p>


<font size="4" color="black"> Now we are getting to the part that will differ for most people. We've inspected the data and found some interesting correlations and nuggets of wisdom. However, choosing which variables to move forward with is an art not a science and every data scientist will do something slightly differently.

<font size="4" color="black">I've decided to move forward with the following variables:</font>
    
*   <font size="4" color="black">'Pclass' --- I've separated the classes into separate variables (P1,P2,P3)</font>
*   <font size="4" color="black">'Sex'    --- The sex variable is likely to inform the model a lot based on the correlations</font>
*   <font size="4" color="black">'Age'    --- Age seems to be an important variable but I will normalise it and clear all NAs</font>
*   <font size="4" color="black">'SibSp'  --- The siblings tell us something about traveling with family which may be helpful</font>
*   <font size="4" color="black">'norm_sibsp' --- Here I've normalised the sibling values</font>
*   <font size="4" color="black">'Parch' --- Likewise the parentage tells us that people are in groups</font>
*   <font size="4" color="black">'Fare' --- Fare tells us the price of the ticket and is therefore indicative of the wealth of the patron</font>
*   <font size="4" color="black">'norm_fare' --- I've included the normalised price of ticket with NAs removed</font>
*   <font size="4" color="black">'cabin_multiple' --- If the patron had multiple cabins it tells us something about their demographic</font>
*   <font size="4" color="black">'numeric_ticket' --- The ticket number may be informative about the location of the cabin</font>
*   <font size="4" color="black">'Embarked' --- Perhaps the location they embarked from may relate to their cabin location or wealth</font>

In [ ]:

# split up the cabin multiple string using quick expression
all_data['cabin_multiple'] = all_data.Cabin.apply(lambda x: 0 if pd.isna(x) else len(x.split(' ')))

# split the tickets into number tickets and letter tickets
all_data['numeric_ticket'] = all_data.Ticket.apply(lambda x: 1 if x.isnumeric() else 0) # simply binary if they had numbers
all_data['ticket_letters'] = all_data.Ticket.apply(lambda x: ''.join(x.split(' ')[:-1]).replace('.','').replace('/','').lower() if len(x.split(' ')[:-1]) > 0 else 0)

# extract name titles from the beginning of names
all_data.drop(['Name'],axis=1)
#all_data['name_title'] = all_data.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())

# fill in missing data with median = both age and fare were non-parametric and had very skewed responses so median is more appropriate
all_data.Age = all_data.Age.fillna(train_data.Age.median())
all_data.Fare = all_data.Fare.fillna(train_data.Fare.median())

# remove all the empty values from embarked
all_data.dropna(subset=['Embarked'],inplace = False)

# normalising the sibling and fare data with a log transformation
all_data['norm_sibsp'] = np.log(all_data.SibSp+1)
all_data['norm_fare'] = np.log(all_data.Fare+1)



In [ ]:
plt.figure(figsize=[16,12])

sp_norm_data = pd.get_dummies(all_data[['norm_sibsp','SibSp']])
plt.subplot(231)
sns.histplot(data=sp_norm_data,multiple="stack",kde=True,shrink=1.2)

fare_norm_data = pd.get_dummies(all_data[['norm_fare','Fare']])
plt.subplot(232)
sns.histplot(data=fare_norm_data,multiple="stack",kde=True)

sns.histplot()

In [ ]:
# here we are looking at Pclass vs a selection of factors.
fig, (axis1,axis2,axis3,axis4) = plt.subplots(1,4,figsize=(18,14))

sns.boxplot(x='Pclass',y='Fare',hue='Survived',data=all_data,ax=axis1)
axis1.set_title('Pclass vs Fare Survival Comparison')

sns.barplot(x='Pclass',y='Survived',hue='Sex',data=all_data,ax=axis2,orient = "v")
axis2.set_title('Pclass vs Sex Survival Comparison')

sns.boxplot(x= 'Pclass',y='SibSp',hue = 'Survived',data=all_data,ax=axis3)
axis3.set_title('Pclass vs Siblings Survival Comparison')

sns.violinplot(x='Pclass',y='Age',hue='Survived',data=all_data,split=True,ax=axis4)
axis4.set_title('Pclass vs Age Survival Comparison')

In [ ]:
#histogram comparison of sex, class, and age by survival
sca = sns.FacetGrid(all_data, row = 'Sex', col = 'Pclass', hue = 'Survived')
sca.map(plt.hist, 'Age', alpha = .55)
sca.add_legend()

In [ ]:
#correlation heatmap of dataset
def corr_hm(df):
    _ , axis1 = plt.subplots(figsize =(14, 12))
    colormap = sns.diverging_palette(220, 10, as_cmap = True)
    
    _ = sns.heatmap(
        df.corr(method = 'spearman'), 
        cmap = colormap,
        square=True, 
        cbar_kws={'shrink':.9 }, 
        ax=axis1,
        linewidths=0.1,vmax=1.0, linecolor='grey',
        annot=True,
        annot_kws={'fontsize':14 }
    )
    
    plt.title('Spearman Correlation of Features', y=1.05, size=15)

corr_hm(all_data)

## <p style="background-color:#90EE90;color:black;font-size:30px;text-align:center;border-radius:12px 10px;border:2px;"> 3. Transforming the Data and Creating Dummies </p>



In [ ]:
# transforming the class into a string
all_data.Pclass = all_data.Pclass.astype(str)

# create dummies for analysis
all_dummies = pd.get_dummies(all_data[['Pclass','Sex','Age','SibSp','Fare','norm_sibsp','norm_fare','Parch','Embarked',
                                     'cabin_multiple','numeric_ticket','train','Survived']])

# separate the data back into training and testing sets
X_train = all_dummies[all_dummies.train == 1].drop(['train','Survived'], axis = 1)
X_test = all_dummies[all_dummies.train == 0].drop(['train'], axis = 1)

y_train = all_data[all_data.train==1].Survived
y_train.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

# now we are transforming the values into something scaled using StandardScaler from sklearn
all_dummies_scaled = all_dummies.copy()
all_dummies_scaled[['Age','SibSp','Parch','norm_fare']] = scale.fit_transform(all_dummies_scaled[['Age','SibSp','Parch','norm_fare']])

# we are then separating the training and testing sets out with the scaled data
X_train_scaled = all_dummies_scaled[all_dummies_scaled.train == 1].drop(['train','Survived'], axis = 1)
X_test_scaled = all_dummies_scaled[all_dummies_scaled.train == 0].drop(['train'], axis = 1)

y_train = all_data[all_data.train == 1].Survived

## <p style="background-color:#90EE90;color:black;font-size:30px;text-align:center;border-radius:12px 10px;border:2px;"> 4. Building the Model </p>


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split

<font size="6" color="black"> Model Tuning

<font size="4" color="black"> For the models we are running a Random Forest Classifier and a Support Vector Machine.
    I have run a GridSearch with different tuning parameters to identify the best setup to use for these classifiers.
    For Random Forest that was the following:
    
 <font size="4" color="black">   
     
    bootstrap: True
     
    criterion: gini
     
    max_depth: 15
     
    max_features: 10
     
    min_samples_leaf: 3
     
    min_samples_split: 2
     
    n_estimators: 550
     

</font>

In [ ]:
rf = RandomForestClassifier(random_state = 1,bootstrap=True,criterion='gini',max_depth=15,max_features=10,min_samples_leaf=3,min_samples_split=2,
                           n_estimators=550)
best_clf_rf = rf.fit(X_train_scaled,y_train)

<font size="4" color="black"> I performed the same GridSearch for the SVM and found the following best parameters:
    
 <font size="4" color="black">   
     
    C: 1
     
    gamma: 0.1
     
    kernel: rbf
     

In [ ]:
svc = SVC(probability = True,C=1.0,kernel="rbf",gamma=0.1)
best_clf_svc = svc.fit(X_train_scaled,y_train)

<font size="4" color="black"> Here we are extracting the feature importance for the two models. The plan is to take the combined top10 features of both models to use as our features for the deep learning.
    
 <font size="4" color="black">  

In [ ]:

perm_importance = permutation_importance(best_clf_rf, X_train_scaled, y_train)
feat_importances_rf = pd.Series(perm_importance.importances_mean,index=X_train_scaled.columns)

feat_importances_rf_plt = feat_importances_rf.nlargest(10).plot(kind='barh')

In [ ]:
perm_importance = permutation_importance(best_clf_svc, X_train_scaled, y_train)
feat_importances_svm = pd.Series(perm_importance.importances_mean,index=X_train_scaled.columns)

feat_importances_svm_plt = feat_importances_svm.nlargest(10).plot(kind='barh')

In [ ]:
# find the common features for deep learning
a=feat_importances_svm.nlargest(10).index
b=feat_importances_rf.nlargest(10).index
df=pd.DataFrame([a,b]).transpose()#

df = df[0].append(df[1]).drop_duplicates('first')

X_train_DL = X_train_scaled[df]
X_test_DL = X_test_scaled[df]

y_train_DL = y_train

dense_neuron = X_train_DL.shape[1]

<font size="4" color="black"> I ran the deep learning model looping through the 8 different optimizers to find the version with the highest accuracy/lowest loss
    
    
<table style="border-collapse: collapse;font-size: 15px; width:800px;">
  <tr>
    <th style="background-color:#D3DBDD;">Optimizer </th>
    <th style="background-color:#D3DBDD;">Loss</th>
    <th style="background-color:#D3DBDD;">Accuracy Percentage</th>
  </tr>
  <tr>
    <td>SGD</td>
    <td>0.125</td>
    <td>83.38%</td>
  </tr>
  <tr>
      <td><b>RMSprop</b> </td>
    <td><b>0.0976</b></td>
    <td><b>87.54%</b></td>
  </tr>
  <tr>
    <td>Adam </td>
    <td>0.108 </td>
    <td>86.083%</td>
  </tr>
  <tr>
    <td>Adadelta </td>
    <td>0.264</td>
    <td>37.59%</td>
  </tr>
  <tr>
    <td>Adagrad </td>
    <td>0.133 </td>
    <td>82.37%</td>
  </tr>
  <tr>
    <td>Adamax</td>
    <td>0.110</td>
    <td>85.29%</td>
  </tr>
  <tr>
    <td>Nadam</td>
    <td>0.106</td>
    <td>86.08%</td>
  </tr>
  <tr>
    <td>Ftrl</td>
    <td>0.249 </td>
    <td>61.61%</td>
  </tr>
</table>
 
<font size="4" color="black"> As we can see, RMRprop gave the best results, so we will use that optimizer moving forward

In [ ]:
# separating the training data into train/test splits for use in our model fitting
XDL_train,XDL_test,yDL_train,yDL_test=train_test_split(X_train_DL,y_train,test_size=0.2,random_state=1234)

In [ ]:
# here we are defining some parameters to use in our model

from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import KFold
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tqdm.notebook import tqdm
from sklearn.utils import shuffle
import tensorflow.keras.backend as K

COLS = list(XDL_train.columns) # defining the columns

batch_size=128
epoch = 500
mae = pd.DataFrame
kFold = KFold(n_splits=10,
              shuffle=True) # defining the number of cross-validation kfolds we want to perform

inputs = np.concatenate((XDL_train,
                         XDL_test),
                        axis=0) # combining the test/train data

targets = np.concatenate((yDL_train,
                          yDL_test),
                         axis=0)

<font size="4" color="black"> Brace yourself, this is a wall of text. I could have subdivided this into several functions but it all works and runs well. Perhaps I can clean it up a bit another time. But for now lets consider what this model is achieving:
    
* <font size="3" color="black">First we are defining a table that will output our mean absolute error scores.
    
* <font size="3" color="black">Next we are collecting the results into lists that we can use for printing.
    
* <font size="3" color="black">The model then enters into an outer for loop that will loop over the number of folds that we want to run (defined above).
    
* <font size="3" color="black">Within this loop we will re-define the model on every run, compile the model using our predefined standards and fit the model.
    
* <font size="3" color="black">Before we fit, we are creating several callbacks that can be used during the run:
    
   &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;        1.<font size="3" color="black"> A callback that will stop the current run if certain amount of progression has passed without improvement.
       
  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;         2.<font size="3" color="black"> A callback that saves the best weights based on improvements across each epoch.
      
* <font size="3" color="black">The data is split manually based on the kFolds function defined above. Each iteration of the outer loop will use a different subsection of the data for test and validation.
    
* <font size="3" color="black">After the data are fit to the model, we want to extract the feature importance much like we have done above.
    
* <font size="3" color="black">For this to work first we must define a baseline score, here calculated by defining the out of function predictions based on all the valid data and then taking the absolute mean of that score divided by the valid list.
    
* <font size="3" color="black">After this we enter into an inner loop which does the same thing for each feature but shuffles the column each time.
    
* <font size="3" color="black">The last part of the function is just dealing with the outputs </font>   

In [ ]:
def DLmodel(inputs,targets,columns,batch_size,epochs,kFold):
    results_table = {}
    acc_per_fold = [] # variable to collect the accuracy for printing and saving
    loss_per_fold = [] # variable to collect the loss for printing and saving
    i = 0
    fold_no = 1
    for fold,(train_idx,test_idx) in enumerate(kFold.split(inputs,targets)):
    
        K.clear_session()
        
        print('-'*15, '>', f'Fold {fold+1}', '<', '-'*15)
        X_train, X_valid = inputs[train_idx], inputs[test_idx]
        y_train, y_valid = targets[train_idx], targets[test_idx]
        checkpoint_filepath = f"folds{fold}.hdf5"
    
        #################################################################
        # creating the keras model
        model = keras.Sequential([
            layers.BatchNormalization(),
            layers.Dense(32,
                         activation = 'relu',
                         input_shape=([dense_neuron])),
            layers.Dropout(0.3),
            layers.BatchNormalization(),
            layers.Dense(32,
                         activation = 'relu'),
            layers.BatchNormalization(),
            layers.Dense(32,
                         activation = 'relu'),
            layers.Dropout(0.3),
            layers.Dense(1,
                         activation = 'sigmoid')
        ])

        model.compile(
            optimizer = 'RMSprop',
            loss = 'mae',
            metrics=['accuracy']
    
        )
        #################################################################
        # Generate a print
        print('------------------------------------------------------------------------')
        print(f'Training for fold {fold_no} ...')
    
        #################################################################
        # create the early stopping and saving functionality
        early_stopping = keras.callbacks.EarlyStopping(
            patience = 100,
            min_delta = 0.001,
            restore_best_weights=True,
        )
        save_best = keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_filepath,
            save_weights_only=False,
            monitor='val_accuracy',
            mode='max',
            verbose=0,
            save_best_only=True)

        # fit the model
        history = model.fit(
            X_train,
            y_train,
            validation_data=(X_valid,
                             y_valid),    
            batch_size=batch_size,
            epochs = epochs,
            callbacks=[early_stopping,
                       save_best],
            verbose = False,
        )
        scores = model.evaluate(X_valid,
                                y_valid,
                                verbose=0)
        print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]};{model.metrics_names[1]} of {scores[1]*100}%')
        print(f'optimizer: RMSprop')
        acc_per_fold.append(scores[1]*100)
        loss_per_fold.append(scores[0])   

        #################################################################
        # Increase fold number

        
        print('Calculating feature importance...')
            
        # compute baseline (no shuffle)
        oof_preds = model.predict(X_valid,
                                  verbose=0).squeeze() 
        baseline_mae = np.mean(np.abs(oof_preds-y_valid ))
    
        for col in columns:
            if not col in results_table.keys():
                results_table[col] = []
                results_table['baseline'] = []
       
        results_table['baseline'].append(baseline_mae)
    
        ################################################################# 
        # add col names before loops
    

        # defining a function to take care of the inner loop calculations
        for k in tqdm(range(len(columns))):
            # shuffle feature 
            save_col = X_valid[:,k].copy()
            np.random.shuffle(X_valid[:,k])
            col_name = columns[k]
            # computer oof mae with feature k shuffled
            oof_preds = model.predict(X_valid,
                                        verbose=0).squeeze()

            results_table[col_name].append(np.mean(np.abs(oof_preds-y_valid)))
            X_valid[:,k] = save_col
    
        
    
        #################################################################
        # display feature importance
        
        df = pd.DataFrame.from_dict(results_table)
        df = df.iloc[fold].sort_values(ascending=True)
        plt.figure(figsize=(5,6))
        plt.barh(np.arange(len(columns)+1),
                 df.values)
        plt.yticks(np.arange(len(columns)+1),
                   df.index)
        plt.title(f'K-Fold {fold+1} Feature Importance',
                  size=16)
        plt.ylim((-1,len(columns)+1))
        plt.plot([baseline_mae,
                  baseline_mae],
                 [-1,len(columns)+1],
                 '--', color='red',
                 label=f'Baseline OOF\nMAE={baseline_mae:.3f}')
        plt.xlabel(f'Fold {fold+1} OOF MAE with feature permuted',
                   size=14)
        plt.ylabel('Feature',
                   size=14)
        plt.legend()
        plt.show()
                               
        # SAVE FEATURE IMPORTANCE
        df.to_csv(f'feature_importance_fold_{fold+1}.csv',
                  index=False)
    
        fold_no = fold_no + 1
        i = i+1
    
    #################################################################

    # == Provide average scores ==
    print('------------------------------------------------------------------------')
    print('Score per fold')
    for i in range(0, len(acc_per_fold)):
      print('------------------------------------------------------------------------')
      print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
    print('------------------------------------------------------------------------')
    print('Average scores for all folds:')
    print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
    print(f'> Loss: {np.mean(loss_per_fold)}')
    print('------------------------------------------------------------------------')

    history_df = pd.DataFrame(history.history)
    history_df.loc[:, ['loss', 'val_loss','accuracy','val_accuracy']].plot();
    print("Minimum validation loss: {}".format(history_df['val_loss'].min()))
    pd.DataFrame.from_dict(results_table)
    
    return history, results_table, save_best

In [ ]:
history,results_table, save_best = DLmodel(inputs,targets,COLS,batch_size,epoch,kFold)

<font size="4" color="black"> As we can see from the output, the model is performing pretty well (between 80-90% accuracy). The plan now, which is totally not necessary and is purely for exploration at this point. Is to take the overall best features across all of the folds and take the best 5 and apply it to one more run. Just to see if using less features that are the most informative is able to squeeze out a little more accuracy. Obviously we want to be careful as well to keep an eye on our loss/val_loss scores to make sure we aren't overfitting/underfitting.

In [ ]:
from matplotlib.pyplot import figure

param_data = pd.DataFrame.from_dict(results_table)
mean_param_data = {}
for i in param_data:
    mean_param_data[i] = []
    mean_param_data[i].append(param_data[i].mean())
figure(figsize=(16, 12), dpi=80)
plt.scatter(*zip(*sorted(mean_param_data.items()))) 
plt.show()
mean_param_data = pd.Series(mean_param_data)

<font size="3" color="black"> Here is a simple scatter plot detailing the average feature importance across all the folds. We are going to take the best 5 and use it for the final model.

In [ ]:
top5_params = mean_param_data.sort_values(ascending = False)[0:5]
top5COLS = list(top5_params.index)

inputs = np.concatenate((XDL_train[top5COLS],
                         XDL_test[top5COLS]),
                        axis=0) # combining the test/train data

targets = np.concatenate((yDL_train,
                          yDL_test),
                         axis=0)

In [ ]:
best_params,results_table_params, save_best_params = DLmodel(inputs,targets,top5COLS,batch_size,epoch,kFold)

<font size="3" color="black"> Accuracy seems to be staying put, still around 80-90%. Which is interesting in of itself. Even without the other variables we are still able to predict survival likelihood to a very high percentage.

In [ ]:
Xfinal_train = X_train_DL[top5_params.index] # creating the testing set with the finalised features
Xfinal_test = X_test_DL[top5_params.index]
best_final = {}
for i in range(kFold.n_splits): 
    # this loop extracts the accuracy and loss from each of our folds evaluated on the whole.
    # training set rather than using out of sample kFolds. We will then save this into a library and find the very best weights to use.
    # this isn't strictly best practice, but like I said earlier, this is more just for exploration and seeing what we can squeeze out.
    save_best_params.model.load_weights(f'folds{i}.hdf5')
    best_final[f'folds{i}.hdf5'] = []
    best_final[f'folds{i}.hdf5'].append(save_best_params.model.evaluate(Xfinal_train,y_train_DL,verbose=True)[1])

In [ ]:
save_best_params.model.load_weights(max(best_final))
save_best_params.model.evaluate(Xfinal_train,y_train_DL,verbose=True)

## <p style="background-color:#90EE90;color:black;font-size:30px;text-align:center;border-radius:12px 10px;border:2px;"> 4. Model Prediction of Survival </p>

<font size="3" color="black"> So here we are at the moment of truth! We are going to use the testing data with our modified feature inputs to predict survival and submit to the competition!


In [ ]:
from keras import models

print('Generate a prediction')
prediction = np.round(save_best_params.model.predict(Xfinal_test)).astype(int)

print('prediction shape:',prediction.shape)

In [ ]:
final_data = {'PassengerId':test_data.PassengerId,
             'Survived':prediction.transpose(1,0)[0,:]}

submission = pd.DataFrame(final_data)

In [ ]:
submission.to_csv('submission.csv',index=False)